In [ ]:
import pandas as pd
import xlwings as xw
import os
import time

# File paths
source_template = r"C:\Users\141823\OneDrive - Etex Group\Desktop\Etex AU - Finance - Finance\19.Business Systems\Overhead Report\Etex Consolidated- Overhead Report - Template.xlsm"
reference_file = r"C:\Users\141823\OneDrive - Etex Group\Desktop\Etex AU - Finance - Finance\19.Business Systems\Overhead Report\Email List.xlsx"
output_folder = r"C:\Users\141823\OneDrive - Etex Group\Desktop\Etex AU - Finance - Finance\19.Business Systems\Overhead Report\Send\Review"

# Load the reference mapping
df_ref = pd.read_excel(reference_file, sheet_name=0, usecols="B:C", skiprows=1, names=["Workbook", "Sheet Name"])

# Group the sheet names by workbook name
sheet_groups = df_ref.groupby("Workbook")["Sheet Name"].apply(list).to_dict()

# Start xlwings App
app = xw.App(visible=False)

# Load source template workbook ONCE
source_wb = app.books.open(source_template, read_only=True)

total_workbooks = len(sheet_groups)
current_workbook = 0

for workbook_name, sheet_list in sheet_groups.items():
    current_workbook += 1
    dest_path = os.path.join(output_folder, workbook_name)
    if not os.path.exists(dest_path):
        print(f"Workbook not found: {dest_path}")
        continue

    dest_wb = app.books.open(dest_path)

    for sheet_name in sheet_list:
        if sheet_name not in [sheet.name for sheet in source_wb.sheets]:
            print(f"Sheet '{sheet_name}' not found in source template.")
            continue

        source_ws = source_wb.sheets[sheet_name]

        if sheet_name == "Transaction":
            # Replace whole sheet
            if sheet_name in [sheet.name for sheet in dest_wb.sheets]:
                dest_wb.sheets[sheet_name].delete()

            dest_ws = dest_wb.sheets.add(name=sheet_name)

            # Copy entire data
            source_data = source_ws.range('A1').expand()
            dest_ws.range('A1').value = source_data.value

        else:
            # Replace only D6:AA188
            if sheet_name not in [sheet.name for sheet in dest_wb.sheets]:
                print(f"Sheet '{sheet_name}' not found in destination workbook.")
                continue

            dest_ws = dest_wb.sheets[sheet_name]
            source_data = source_ws.range('D6:AA188')
            dest_ws.range('D6:AA188').value = source_data.value  # values only

    dest_wb.save()
    dest_wb.close()

    # --- Progress bar ---
    progress = (current_workbook / total_workbooks) * 100
    print(f"Progress: {current_workbook}/{total_workbooks} workbooks processed ({progress:.2f}%)")
    time.sleep(0.2)  # (Optional) Small sleep to make the console output nicer

# Clean up
source_wb.close()
app.quit()

print("\n✅ All workbooks updated successfully!")


com_error: (-2147352567, 'Exception occurred.', (0, 'Microsoft Excel', "Microsoft Excel cannot access the file 'C:\\Users\\141823\\OneDrive - Etex Group\\Desktop\\Etex AU - Finance - Finance\\19.Business Systems\\Overhead Report\\Send\\Review\\Overhead Report Mar 2025 - Administration.xlsx'. There are several possible reasons:\n\n• The file name or path does not exist.\n• The file is being used by another program.\n• The workbook you are trying to save has the same name as a currently open workbook.", 'xlmain11.chm', 0, -2146827284), None)